In [1]:
import pandas as pd
import os
import glob
import mygene 
# os.chdir(os.path.join('tissue_cocoputs','data'))

# print(os.listdir()
# )
# human_cds = pd.read_csv('o586358-Human_CDS_Bicod.tsv')

# human_cds

def subset_and_write(df,col_list,fn):
    df = df.set_index('transcript_id')
    df = df[col_list]
    df.to_csv(fn)

def create_tissue_directory(sample_data, tissues):
    sample_dir = {}
    for tissue in tissues:
        sample_dir[tissue] = sample_data.loc[sample_data['SMTSD'] == tissue]['SAMPID'].values.tolist()
    return(sample_dir)



def generate_bicodon_table(refseqs,weights,file_id):

    bcds_sorted = bcds.loc[refseqs,:]
    bcds_weighted = (bcds_sorted.T * weights).sum(axis=1)
    bcds_weighted.index = bcds_weighted.index.str.upper()
    bcds_weighted.to_frame().transpose().to_csv(
        'bicodon_tables/GTEx_{}_{}_v7.tsv'.format(tissue,file_id), sep='\t', index=False)
    print('Generated Bicodon Table')
    return(bcds_weighted)


def generate_codon_table(refseqs,weights,file_id):

    cds_sorted = cds.loc[refseqs,:]
    cds_weighted = (cds_sorted.T * weights).sum(axis=1)

    # output codon freq table
    output = pd.read_csv('/grid/home/nbourgeois/codonOpt/references/codon_usage.getex.txt',sep='\t')
    output = output.drop(columns='Fraction')
    output.loc[:,'Number'] = cds_weighted.values.astype(int)
    output.loc[:,'Frequency'] = round(output['Number'] / sum(output['Number']) * 1000,1)
    output.to_csv('codon_tables/GTEx_{}_{}_v7.tsv'.format(tissue,file_id), sep='\t', index=False)
    print('Generated Codon Table')

In [2]:

ref_dir = os.path.join(os.getcwd(),'..','references')
os.chdir('/grid/home/nbourgeois/codonOpt/tissue_cocoputs/data')

# Original Data
gtex_data = pd.read_csv('TPM/GTEx_Lung_TPM_orig.tsv',sep='\t')
gtex_samp_data= pd.read_csv('ref/GTEx_v7_Annotations_SampleAttributesDS.txt',sep='\t')




In [3]:
gtex_samp_data = gtex_samp_data.loc[gtex_samp_data['SAMPID'].isin(gtex_data.columns)]

tissues = gtex_samp_data['SMTSD'].unique().tolist()
sample_dir = create_tissue_directory(gtex_samp_data,tissues)



In [4]:
bcds = pd.read_csv('o586358-Human_CDS_Bicod.tsv',sep='\t')
bcds = bcds.drop(columns = ['Protein ID', 'Accession', 'Division', 'Assembly', 'Taxid',
                        'Species', 'Organelle', '# Codon Pairs', 'GC%', 'GC1%', 'GC2%', 'GC3%','Unnamed: 4109'])
bcds = bcds.groupby('Gene ID').sum()

# convert protein info to ensmbl transcript info



In [5]:
bcds

,tttttt,tttttc,ttttta,tttttg,tttctt,tttctc,tttcta,tttctg,tttatt,tttatc,...,gggcga,gggcgg,gggagt,gggagc,gggaga,gggagg,gggggt,gggggc,ggggga,gggggg
Gene ID,,,,,,,,,,,,,,,,,,,,,
A1BG,0,0,0,0,0,1,0,2,0,0,...,0,0,0,0,0,0,0,1,0,0
A1CF,0,0,0,0,0,0,0,0,11,0,...,0,0,0,0,0,0,0,12,0,0
A2M,0,0,0,0,0,0,0,0,0,0,...,0,5,0,5,0,0,0,0,0,0
A2ML1,0,7,0,0,6,0,0,6,0,0,...,0,0,0,7,0,0,0,14,0,0
A3GALT2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11A,0,6,0,0,0,10,0,0,0,0,...,0,0,0,0,0,0,0,10,0,0
ZYG11B,0,2,0,0,0,0,0,0,3,0,...,0,0,0,0,3,2,0,0,0,0
ZYX,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,7,0,0


In [6]:
mg = mygene.MyGeneInfo()
geneSyms = mg.querymany(bcds.index.values, scopes='symbol',
                        fields='ensembl.gene', species='human')



querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
querying 19001-20000...done.
querying 20001-20239...done.
Finished.
430 input query terms found dup hits:
	[('ABO', 3), ('AGAP11', 2), ('ALG1L', 2), ('ARMCX5-GPRASP2', 2), ('ATP6AP1L', 2), ('BAGE2', 2), ('BO
336 input query terms found no hit:
	['ACPP', 'ADPRHL2', 'ANKRD20A2', 'ANKRD20A3', 'ANKRD20A4', 'ARMC4', 'ATP5MD', 'ATP5MPL', 'BHMG1', 'C
Pass "returnall=True" to return complete lists of duplicate or missing query terms.


In [7]:
tissue = 'Lung'

# subset to specific tissue
cols = ['transcript_id','gene_id'] + sample_dir[tissue]
gtex_data_tissue = gtex_data[cols]
tissue = tissue.replace(" - ", "_").replace(" ", "_").replace("_(BA9)", "")


In [8]:
# reformat
gtex_data_tissue['gene_id'] = gtex_data_tissue['gene_id'].str.split('.',expand=True)[0]
gtex_data_tissue_summed = gtex_data_tissue.groupby('gene_id').sum()



/data/software/anaconda3/envs/bfx-codon/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
# get average per transcript
gtex_avg = gtex_data_tissue_summed.mean(axis=1)




In [10]:
def get_weights(geneSyms, gtex_avg):

    def append_results(gene):
        if gene in gtex_avg.index:
            weight = gtex_avg[gene]
            symbols.append(gdict['query'])
            weights.append(weight)

        
    symbols = []
    weights = []
    counter = 0
    for gdict in geneSyms:
        if 'ensembl' in gdict:
            if len(gdict['ensembl']) == 1:
                gene = gdict['ensembl']['gene']
                append_results(gene)

            else:
                for ensembl in gdict['ensembl']:
                    gene = ensembl['gene']
                    append_results(gene)

        counter +=1    

    return(symbols,weights)


symbols,weights = get_weights(geneSyms, gtex_avg)

In [11]:
symbols = []
weights = []





In [13]:
bicod_tab = generate_bicodon_table(symbols,weights,'bicodon_test')

Generated Bicodon Table


In [14]:
bicod_tab

TTTTTT    0.0
TTTTTC    0.0
TTTTTA    0.0
TTTTTG    0.0
TTTCTT    0.0
         ... 
GGGAGG    0.0
GGGGGT    0.0
GGGGGC    0.0
GGGGGA    0.0
GGGGGG    0.0
Length: 4096, dtype: float64

In [15]:
bcds_sorted = bcds.loc[symbols,:]
bcds_weighted = (bcds_sorted.T * weights).sum(axis=1)
bcds_weighted.index = bcds_weighted.index.str.upper()
bcds_weighted.to_frame()

,0
TTTTTT,0.0
TTTTTC,0.0
TTTTTA,0.0
TTTTTG,0.0
TTTCTT,0.0
...,...
GGGAGG,0.0
GGGGGT,0.0
GGGGGC,0.0
GGGGGA,0.0


In [16]:
weights[0:10]

[]

In [17]:
bcds.loc[refseqs,:].T.sum(axis=1)

NameError: name 'refseqs' is not defined

In [ ]:
(bcds_sorted.T * weights).sum(axis=1).sum()

In [ ]:
pd.read_csv('../../references/CoCoPUTs_codon_usage/bicodon_usage/Liver.bicodon.txt',sep='\t').T.sum()